# GBDT - LightGBM

In [2]:
%load_ext autoreload
%autoreload 2
    
import sys
import os
import numpy as np
import pandas as pd
import lightgbm
import pickle
import json
import functools
from os import listdir
from os.path import isfile, join

from NILib import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def train_gradient_boosting_baseline( train_file, valid_file, test_file,
                                output_model_file):
    
    exp = pd.DataFrame(columns=['num_trees', 'learning_rate', 'num_leaves', 'best_round', 'avg_binary_log_loss'])
    
    # load train/valid/test
    train, valid, test, cat_fx = load_atk_train_valid_test(train_file, valid_file, test_file)
    
    assert "instance_id" not in train.columns.values, "Wrong training set file for GBDT"

    # get index of categorical features 
    cat_fx = np.where(train.columns.isin(cat_fx))[0]
    cat_fx = list([int(x) for x in cat_fx])  
    print ("CatFX:", train.columns.values[cat_fx])
    

    for num_trees in [500]:
        best_model = None
        best_info = None
        best_loss = np.inf
        for learning_rate in [0.1]: #[0.01, 0.05, 0.1]:
            for num_leaves in [16]: #[8, 16, 24]:
                # datasets
                lgbm_train = lightgbm.Dataset(data=train.iloc[:,:-1].values, 
                                              label=train.iloc[:,-1].values,
                                              categorical_feature = cat_fx)

                lgbm_valid = lightgbm.Dataset(data=valid.iloc[:,:-1].values, 
                                              label=valid.iloc[:,-1].values,
                                              categorical_feature = cat_fx)

                # run train
                lgbm_params = { 'learning_rate': learning_rate, 
                                'num_leaves': num_leaves} 
                lgbm_info = {}
                lgbm_model = lightgbm.train(lgbm_params, lgbm_train, 
                                            num_boost_round = num_trees,
                                            fobj            = optimize_log_loss, 
                                            feval           = avg_log_loss,
                                            evals_result    = lgbm_info,
                                            valid_sets      = [lgbm_train, lgbm_valid], 
                                            valid_names     = ['train', 'valid'],
                                            verbose_eval    = 5)
                
                if np.min(lgbm_info['valid']['avg_binary_log_loss']) < best_loss:
                    best_model = lgbm_model
                    best_info = lgbm_info
                    best_loss = np.min(lgbm_info['valid']['avg_binary_log_loss'])
                    best_info['num_trees'] = num_trees
                    best_info['learning_rate'] = learning_rate
                    best_info['num_leaves'] = num_leaves
                    
                    
                best_valid_iter = np.argmin(lgbm_info['valid']['avg_binary_log_loss'])
                
                # update experimental results
                exp = exp.append({'num_trees': num_trees, 
                                  'learning_rate':learning_rate,
                                  'num_leaves':num_leaves, 
                                  'best_round':best_valid_iter+1, 
                                  'avg_binary_log_loss':lgbm_info['valid']['avg_binary_log_loss'][best_valid_iter]},
                                 ignore_index=True)
                
        
        # save file
        best_valid_iter = np.argmin(best_info['valid']['avg_binary_log_loss'])

        model_file_name = "{:s}_T{:d}_S{:04d}_L{:d}_R{:d}.model".format(output_model_file,
                                                                        best_info['num_trees'],
                                                                        int(best_info['learning_rate']*1000),
                                                                        best_info['num_leaves'],
                                                                        best_valid_iter + 1
                                                                       )
        
        best_model.save_model(model_file_name)
        print ("Model saved to", model_file_name)
        
        best_model = lightgbm.Booster(model_file=model_file_name)
        print ("Check valid score:", avg_log_loss(preds=best_model.predict(valid.iloc[:,:-1].values),
                                                  train_data=lgbm_valid))

    
    return exp

In [4]:
# enable/disable LGBM Baseline
if True:
    experiments = train_gradient_boosting_baseline("../data/census/train_ori.csv.bz2",
                                                     "../data/census/valid_ori.csv.bz2",
                                                     "../data/census/test_ori.csv.bz2",
                                                     "../out/models/std_gbdt_census")  

    experiments.to_csv('../out/models/std_gbdt_census.csv', index=False)

    print (experiments)

Loading pre-processed files...
CatFX: ['workclass' 'marital_status' 'occupation' 'relationship' 'race' 'sex'
 'native_country']


/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/lucchese/.local/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[5]	train's avg_binary_log_loss: 0.5195	valid's avg_binary_log_loss: 0.520269
[10]	train's avg_binary_log_loss: 0.442694	valid's avg_binary_log_loss: 0.443485
[15]	train's avg_binary_log_loss: 0.404239	valid's avg_binary_log_loss: 0.404565
[20]	train's avg_binary_log_loss: 0.383888	valid's avg_binary_log_loss: 0.384591
[25]	train's avg_binary_log_loss: 0.372059	valid's avg_binary_log_loss: 0.372916
[30]	train's avg_binary_log_loss: 0.364054	valid's avg_binary_log_loss: 0.365226
[35]	train's avg_binary_log_loss: 0.359023	valid's avg_binary_log_loss: 0.360431
[40]	train's avg_binary_log_loss: 0.354723	valid's avg_binary_log_loss: 0.35629
[45]	train's avg_binary_log_loss: 0.346831	valid's avg_binary_log_loss: 0.349523
[50]	train's avg_binary_log_loss: 0.332285	valid's avg_binary_log_loss: 0.336143
[55]	train's avg_binary_log_loss: 0.326506	valid's avg_binary_log_loss: 0.331138
[60]	train's avg_binary_log_loss: 0.318182	valid's avg_binary_log_loss: 0.32389
[65]	train's avg_binary_log_loss: